In [2]:

"""
문학광장 사이트를 크롤링하기 위해 사용됨
https://teen.munjang.or.kr/archives/category/write/life
"""

import requests
from bs4 import BeautifulSoup
import os
from urllib.request import urlopen
from urllib.request import Request
from tqdm import tqdm

import pandas as pd


# 폴더 생성
save_folder_path = "/workspace/home/uglee/Projects/title_extraction/datasets/raw_poem_datasets"

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
            print("Folder made")
    except OSError:
        print("Error")

createFolder(save_folder_path)

# https://teen.munjang.or.kr/archives/category/write/life/page/2

# URL
TEEN_URL_LIST = [
    'https://teen.munjang.or.kr/archives/category/write/poetry/page/' + \
    str(i + 1)
    for i in range(2261)
]

TEEN_URL_LIST[0]

'https://teen.munjang.or.kr/archives/category/write/poetry/page/1'

In [3]:
poem_link_list = []

# 번호별 페이지
for idx, teen_url in enumerate(tqdm(TEEN_URL_LIST)):

    # gangwon_url 은 각 페이지별 url

    # page 하나 가져오기
    req = Request(teen_url, headers={'User-Agent': 'Mozila/5.0'})
    webpage = urlopen(req)
    soup = BeautifulSoup(webpage, 'html.parser')

    # 한 페이지 내에서만 찾기
    for i, tag in enumerate(soup.find_all('a', attrs={'rel': 'bookmark'})):

        # 만약 태그 안에 [공지]가 있다면 링크 저장하지 않기
        if "[공지]" in tag.text:
            continue

        poem_link = tag.attrs["href"]

        poem_link_list.append(poem_link)

print(poem_link_list[0])

100%|██████████| 2261/2261 [48:47<00:00,  1.29s/it]

https://teen.munjang.or.kr/archives/136689


In [4]:
import re

pre_folder_path = "/workspace/home/uglee/Projects/title_extraction/datasets/pre_poem_datasets"

# 각 essay 별로 접근
for idx, poem_link in enumerate(tqdm(poem_link_list)):

    req = Request(poem_link, headers={'User-Agent': 'Mozila/5.0'})
    webpage = urlopen(req)
    soup = BeautifulSoup(webpage, 'html.parser')

    title = soup.find('h1', 'entry-title').text
    #title_list.append(title)

    content = soup.find('div', 'entry-content').text
    #content_list.append(content)

     # \t 전처리
    title = title.replace("\t", "")
    title = title.replace("\n", " ")
    title = re.sub(r'\([^)]*\)', '', title)
    title = re.sub(r'\[[^)]*\]', '', title)
    title = re.sub(r'[^\w\s]', '', title)

    content = content.replace("\t", "")
    content = content.replace("\n", " ")

    title_content = title + "\t" + content

    tsv_file_name = "teen_poem_" + str(idx) + ".tsv"

    try:
        with open(os.path.join(pre_folder_path, tsv_file_name), 'w', encoding='utf-8') as f:
            f.write(title_content)
    except:
        continue


100%|██████████| 22597/22597 [4:27:53<00:00,  1.41it/s]  
